# Lab-10-4-2 ImageFolder2

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [7]:
trans = transforms.Compose([
    transforms.ToTensor()  
])
train_data = torchvision.datasets.ImageFolder(root = './custom_data/train_data', transform = trans)

In [8]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers = 2)

In [24]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)     
        out = self.layer3(out)
        return out

In [25]:
# testing
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [26]:
optimizer = optim.Adam(net.parameters(), lr = 0.00001)
loss_func = nn.CrossEntropyLoss().to(device)

In [57]:
total_batch = len(data_loader)
epochs = 7
for epoch in range(epochs):
    avg_cost =0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

[Epoch:1] cost = 0.11371976137161255
[Epoch:2] cost = 0.08363521099090576
[Epoch:3] cost = 0.06464041024446487
[Epoch:4] cost = 0.05045540630817413
[Epoch:5] cost = 0.04171820729970932
[Epoch:6] cost = 0.033990878611803055
[Epoch:7] cost = 0.028833476826548576
Learning Finished!


In [36]:
torch.save(net.state_dict(), "./model/model.pth")

In [37]:
new_net = CNN().to(device)

In [38]:
new_net.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [42]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([ 0.0842,  0.0798, -0.0864, -0.0004,  0.0170], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([ 0.0842,  0.0798, -0.0864, -0.0004,  0.0170], device='cuda:0',
       grad_fn=<SelectBackward0>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

In [45]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform = trans)

In [54]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [58]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imags, label = data
        imgs = imags.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
